# Introduction to Pandas

**pandas** is a Python package providing fast, flexible, and expressive data structures designed to work with *relational* or *labeled* data both. It is a fundamental high-level building block for doing practical, real world data analysis in Python. 

pandas is well suited for:

- Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
- Ordered and unordered (not necessarily fixed-frequency) time series data.
- Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels
- Any other form of observational / statistical data sets. The data actually need not be labeled at all to be placed into a pandas data structure


Key features:
    
- Easy handling of **missing data**
- **Size mutability**: columns can be inserted and deleted from DataFrame and higher dimensional objects
- Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the data can be aligned automatically
- Powerful, flexible **group by functionality** to perform split-apply-combine operations on data sets
- Intelligent label-based **slicing, fancy indexing, and subsetting** of large data sets
- Intuitive **merging and joining** data sets
- Flexible **reshaping and pivoting** of data sets
- **Hierarchical labeling** of axes
- Robust **IO tools** for loading data from flat files, Excel files, databases, and HDF5
- **Time series functionality**: date range generation and frequency conversion, moving window statistics, moving window linear regressions, date shifting and lagging, etc.

In [1]:
import pandas as pd
import numpy as np

## Pandas Data Structures

### Series

A **Series** is a single vector of data (like a NumPy array) with an *index* that labels each element in the vector.

In [2]:
counts = pd.Series([632, 1638, 569, 115])


In [3]:
counts

0     632
1    1638
2     569
3     115
dtype: int64

If an index is not specified, a default sequence of integers is assigned as the index. A NumPy array comprises the values of the `Series`, while the index is a pandas `Index` object.

In [4]:
counts.values

array([ 632, 1638,  569,  115], dtype=int64)

In [5]:
counts.index

RangeIndex(start=0, stop=4, step=1)

We can assign meaningful labels to the index, if they are available:

In [6]:
bacteria = pd.Series([632, 1638, 569, 115], 
    index=['Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes'])

bacteria

Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
dtype: int64

These labels can be used to refer to the values in the `Series`.

In [7]:
bacteria['Actinobacteria']

569

In [8]:
bacteria[[name.endswith('bacteria') for name in bacteria.index]]

Proteobacteria    1638
Actinobacteria     569
dtype: int64

In [9]:
[name.endswith('bacteria') for name in bacteria.index]

[False, True, True, False]

Notice that the indexing operation preserved the association between the values and the corresponding indices.

We can still use positional indexing if we wish.

In [10]:
bacteria[0]

632

We can give both the array of values and the index meaningful labels themselves:

In [11]:
bacteria.name = 'counts'
bacteria.index.name = 'phylum'
bacteria

phylum
Firmicutes         632
Proteobacteria    1638
Actinobacteria     569
Bacteroidetes      115
Name: counts, dtype: int64

NumPy's math functions and other operations can be applied to Series without losing the data structure.

In [12]:
np.log(bacteria)

phylum
Firmicutes        6.448889
Proteobacteria    7.401231
Actinobacteria    6.343880
Bacteroidetes     4.744932
Name: counts, dtype: float64

We can also filter according to the values in the `Series`:

In [13]:
bacteria[bacteria>1000]

phylum
Proteobacteria    1638
Name: counts, dtype: int64

A `Series` can be thought of as an ordered key-value store. In fact, we can create one from a `dict`:

In [14]:
bacteria_dict = {'Firmicutes': 632, 'Proteobacteria': 1638, 'Actinobacteria': 569,
                 'Bacteroidetes': 115}
pd.Series(bacteria_dict)

Actinobacteria     569
Bacteroidetes      115
Firmicutes         632
Proteobacteria    1638
dtype: int64

Notice that the `Series` is created in key-sorted order.

If we pass a custom index to `Series`, it will select the corresponding values from the dict, and treat indices without corrsponding values as missing. Pandas uses the `NaN` (not a number) type for missing values.

In [15]:
bacteria2 = pd.Series(bacteria_dict, 
                      index=['Cyanobacteria','Firmicutes',
                             'Proteobacteria','Actinobacteria'])
bacteria2

Cyanobacteria        NaN
Firmicutes         632.0
Proteobacteria    1638.0
Actinobacteria     569.0
dtype: float64

In [16]:
bacteria2.isnull()

Cyanobacteria      True
Firmicutes        False
Proteobacteria    False
Actinobacteria    False
dtype: bool

Critically, the labels are used to **align data** when used in operations with other Series objects:

In [17]:
bacteria + bacteria2

Actinobacteria    1138.0
Bacteroidetes        NaN
Cyanobacteria        NaN
Firmicutes        1264.0
Proteobacteria    3276.0
dtype: float64

Contrast this with NumPy arrays, where arrays of the same length will combine values element-wise; adding Series combined values with the same label in the resulting series. Notice also that the missing values were propogated by addition.

### DataFrame

Inevitably, we want to be able to store, view and manipulate data that is *multivariate*, where for every index there are multiple fields or columns of data, often of varying data type.

A `DataFrame` is a tabular data structure, encapsulating multiple series like columns in a spreadsheet. Data are stored internally as a 2-dimensional object, but the `DataFrame` allows us to represent and manipulate higher-dimensional data.

In [18]:
data = pd.DataFrame({'value':[632, 1638, 569, 115, 433, 1130, 754, 555],
                     'patient':[1, 1, 1, 1, 2, 2, 2, 2],
                     'phylum':['Firmicutes', 'Proteobacteria', 'Actinobacteria', 
    'Bacteroidetes', 'Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes']})
data

patient          phylum  value
0        1      Firmicutes    632
1        1  Proteobacteria   1638
2        1  Actinobacteria    569
3        1   Bacteroidetes    115
4        2      Firmicutes    433
5        2  Proteobacteria   1130
6        2  Actinobacteria    754
7        2   Bacteroidetes    555

Notice the `DataFrame` is sorted by column name. We can change the order by indexing them in the order we desire:

In [19]:
data[['phylum','value']]

phylum  value
0      Firmicutes    632
1  Proteobacteria   1638
2  Actinobacteria    569
3   Bacteroidetes    115
4      Firmicutes    433
5  Proteobacteria   1130
6  Actinobacteria    754
7   Bacteroidetes    555

A `DataFrame` has a second index, representing the columns:

In [20]:
data.columns

Index([u'patient', u'phylum', u'value'], dtype='object')

The `dtypes` attribute reveals the data type for each column in our DataFrame. 

- `int64` is numeric integer values 
- `object` strings (letters and numbers)
- `float64` floating-point values

In [21]:
data.dtypes

patient     int64
phylum     object
value       int64
dtype: object

If we wish to access columns, we can do so either by dict-like indexing or by attribute:

In [22]:
data['patient']

0    1
1    1
2    1
3    1
4    2
5    2
6    2
7    2
Name: patient, dtype: int64

In [23]:
data.patient

0    1
1    1
2    1
3    1
4    2
5    2
6    2
7    2
Name: patient, dtype: int64

In [24]:
type(data.value)

pandas.core.series.Series

In [25]:
data[['value']]

value
0    632
1   1638
2    569
3    115
4    433
5   1130
6    754
7    555

Notice this is different than with `Series`, where dict-like indexing retrieved a particular element (row). 

If we want access to a row in a `DataFrame`, we index its `loc` attribute.

In [26]:
data.loc[3]

patient                1
phylum     Bacteroidetes
value                115
Name: 3, dtype: object

### Exercise

Try out these commands to see what they return:

- `data.head()`
- `data.tail(3)`
- `data.shape`

An alternative way of initializing a `DataFrame` is with a list of dicts:

In [27]:
data = pd.DataFrame([{'patient': 1, 'phylum': 'Firmicutes', 'value': 632},
                    {'patient': 1, 'phylum': 'Proteobacteria', 'value': 1638},
                    {'patient': 1, 'phylum': 'Actinobacteria', 'value': 569},
                    {'patient': 1, 'phylum': 'Bacteroidetes', 'value': 115},
                    {'patient': 2, 'phylum': 'Firmicutes', 'value': 433},
                    {'patient': 2, 'phylum': 'Proteobacteria', 'value': 1130},
                    {'patient': 2, 'phylum': 'Actinobacteria', 'value': 754},
                    {'patient': 2, 'phylum': 'Bacteroidetes', 'value': 555}])

In [28]:
data

patient          phylum  value
0        1      Firmicutes    632
1        1  Proteobacteria   1638
2        1  Actinobacteria    569
3        1   Bacteroidetes    115
4        2      Firmicutes    433
5        2  Proteobacteria   1130
6        2  Actinobacteria    754
7        2   Bacteroidetes    555

Its important to note that the Series returned when a DataFrame is indexted is merely a **view** on the DataFrame, and not a copy of the data itself. So you must be cautious when manipulating this data:

In [29]:
vals = data.value
vals

0     632
1    1638
2     569
3     115
4     433
5    1130
6     754
7     555
Name: value, dtype: int64

In [30]:
vals[5] = 0
vals

c:\python27\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0     632
1    1638
2     569
3     115
4     433
5       0
6     754
7     555
Name: value, dtype: int64

If we plan on modifying an extracted Series, its a good idea to make a copy.

In [31]:
vals = data.value.copy()
vals[5] = 1000
data

patient          phylum  value
0        1      Firmicutes    632
1        1  Proteobacteria   1638
2        1  Actinobacteria    569
3        1   Bacteroidetes    115
4        2      Firmicutes    433
5        2  Proteobacteria      0
6        2  Actinobacteria    754
7        2   Bacteroidetes    555

We can create or modify columns by assignment:

In [32]:
data.value[[3,4,6]] = [14, 21, 5]
data

c:\python27\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


patient          phylum  value
0        1      Firmicutes    632
1        1  Proteobacteria   1638
2        1  Actinobacteria    569
3        1   Bacteroidetes     14
4        2      Firmicutes     21
5        2  Proteobacteria      0
6        2  Actinobacteria      5
7        2   Bacteroidetes    555

In [33]:
data['year'] = 2013
data

patient          phylum  value  year
0        1      Firmicutes    632  2013
1        1  Proteobacteria   1638  2013
2        1  Actinobacteria    569  2013
3        1   Bacteroidetes     14  2013
4        2      Firmicutes     21  2013
5        2  Proteobacteria      0  2013
6        2  Actinobacteria      5  2013
7        2   Bacteroidetes    555  2013

But note, we cannot use the attribute indexing method to add a new column:

In [34]:
data.treatment = 1
data

patient          phylum  value  year
0        1      Firmicutes    632  2013
1        1  Proteobacteria   1638  2013
2        1  Actinobacteria    569  2013
3        1   Bacteroidetes     14  2013
4        2      Firmicutes     21  2013
5        2  Proteobacteria      0  2013
6        2  Actinobacteria      5  2013
7        2   Bacteroidetes    555  2013

In [35]:
data.treatment

1

### Exercise

From the `data` table above, create an index to return all rows for which the phylum name ends in "bacteria" and the value is greater than 1000.

In [36]:
# Write your answer here 

Specifying a `Series` as a new columns cause its values to be added according to the `DataFrame`'s index:

In [37]:
treatment = pd.Series([0]*4 + [1]*2)
treatment

0    0
1    0
2    0
3    0
4    1
5    1
dtype: int64

In [38]:
data['treatment'] = treatment
data

patient          phylum  value  year  treatment
0        1      Firmicutes    632  2013        0.0
1        1  Proteobacteria   1638  2013        0.0
2        1  Actinobacteria    569  2013        0.0
3        1   Bacteroidetes     14  2013        0.0
4        2      Firmicutes     21  2013        1.0
5        2  Proteobacteria      0  2013        1.0
6        2  Actinobacteria      5  2013        NaN
7        2   Bacteroidetes    555  2013        NaN

Other Python data structures (ones without an index) need to be the same length as the `DataFrame`:

In [39]:
month = ['Jan', 'Feb', 'Mar', 'Apr']
data['month'] = month

ValueError: Length of values does not match length of index

In [40]:
data['month'] = ['Jan']*len(data)
data

patient          phylum  value  year  treatment month
0        1      Firmicutes    632  2013        0.0   Jan
1        1  Proteobacteria   1638  2013        0.0   Jan
2        1  Actinobacteria    569  2013        0.0   Jan
3        1   Bacteroidetes     14  2013        0.0   Jan
4        2      Firmicutes     21  2013        1.0   Jan
5        2  Proteobacteria      0  2013        1.0   Jan
6        2  Actinobacteria      5  2013        NaN   Jan
7        2   Bacteroidetes    555  2013        NaN   Jan

We can use the `drop` method to remove rows or columns, which by default drops rows. We can be explicit by using the `axis` argument:

In [41]:
data.drop('month', axis=1)
data

patient          phylum  value  year  treatment month
0        1      Firmicutes    632  2013        0.0   Jan
1        1  Proteobacteria   1638  2013        0.0   Jan
2        1  Actinobacteria    569  2013        0.0   Jan
3        1   Bacteroidetes     14  2013        0.0   Jan
4        2      Firmicutes     21  2013        1.0   Jan
5        2  Proteobacteria      0  2013        1.0   Jan
6        2  Actinobacteria      5  2013        NaN   Jan
7        2   Bacteroidetes    555  2013        NaN   Jan

We can extract the underlying data as a simple `ndarray` by accessing the `values` attribute:

In [42]:
data.values

array([[1L, 'Firmicutes', 632L, 2013L, 0.0, 'Jan'],
       [1L, 'Proteobacteria', 1638L, 2013L, 0.0, 'Jan'],
       [1L, 'Actinobacteria', 569L, 2013L, 0.0, 'Jan'],
       [1L, 'Bacteroidetes', 14L, 2013L, 0.0, 'Jan'],
       [2L, 'Firmicutes', 21L, 2013L, 1.0, 'Jan'],
       [2L, 'Proteobacteria', 0L, 2013L, 1.0, 'Jan'],
       [2L, 'Actinobacteria', 5L, 2013L, nan, 'Jan'],
       [2L, 'Bacteroidetes', 555L, 2013L, nan, 'Jan']], dtype=object)

Notice that because of the mix of string and integer (and `NaN`) values, the dtype of the array is `object`. The dtype will automatically be chosen to be as general as needed to accomodate all the columns.

In [43]:
df = pd.DataFrame({'foo': [1,2,3], 'bar':[0.4, -1.0, 4.5]})
df.values

array([[ 0.4,  1. ],
       [-1. ,  2. ],
       [ 4.5,  3. ]])

Pandas uses a custom data structure to represent the indices of Series and DataFrames.

In [44]:
data.index

RangeIndex(start=0, stop=8, step=1)

Index objects are immutable:

In [45]:
data.index[0] = 15

TypeError: Index does not support mutable operations

This is so that Index objects can be shared between data structures without fear that they will be changed.

In [46]:
bacteria2.index = bacteria.index

In [47]:
bacteria2

phylum
Firmicutes           NaN
Proteobacteria     632.0
Actinobacteria    1638.0
Bacteroidetes      569.0
dtype: float64

## Pandas Fundamentals

This section introduces the new user to the key functionality of Pandas that is required to use the software effectively.

For some variety, we will leave our digestive tract bacteria behind and employ some baseball data.

In [49]:
baseball = pd.read_csv("baseball.csv", index_col='id')
baseball.head()

player  year  stint team  lg   g  ab  r   h  X2b  ...   rbi   sb  \
id                                                          ...              
88641  womacto01  2006      2  CHN  NL  19  50  6  14    1  ...   2.0  1.0   
88643  schilcu01  2006      1  BOS  AL  31   2  0   1    0  ...   0.0  0.0   
88645  myersmi01  2006      1  NYA  AL  62   0  0   0    0  ...   0.0  0.0   
88649  helliri01  2006      1  MIL  NL  20   3  0   0    0  ...   0.0  0.0   
88650  johnsra05  2006      1  NYA  AL  33   6  0   1    0  ...   0.0  0.0   

        cs  bb   so  ibb  hbp   sh   sf  gidp  
id                                             
88641  1.0   4  4.0  0.0  0.0  3.0  0.0   0.0  
88643  0.0   0  1.0  0.0  0.0  0.0  0.0   0.0  
88645  0.0   0  0.0  0.0  0.0  0.0  0.0   0.0  
88649  0.0   0  2.0  0.0  0.0  0.0  0.0   0.0  
88650  0.0   0  4.0  0.0  0.0  0.0  0.0   0.0  

[5 rows x 22 columns]

Notice that we specified the `id` column as the index, since it appears to be a unique identifier. We could try to create a unique index ourselves by combining `player` and `year`:

In [50]:
player_id = baseball.player + baseball.year.astype(str)
baseball_newind = baseball.copy()
baseball_newind.index = player_id
baseball_newind.head()

player  year  stint team  lg   g  ab  r   h  X2b  ...   rbi  \
womacto012006  womacto01  2006      2  CHN  NL  19  50  6  14    1  ...   2.0   
schilcu012006  schilcu01  2006      1  BOS  AL  31   2  0   1    0  ...   0.0   
myersmi012006  myersmi01  2006      1  NYA  AL  62   0  0   0    0  ...   0.0   
helliri012006  helliri01  2006      1  MIL  NL  20   3  0   0    0  ...   0.0   
johnsra052006  johnsra05  2006      1  NYA  AL  33   6  0   1    0  ...   0.0   

                sb   cs  bb   so  ibb  hbp   sh   sf  gidp  
womacto012006  1.0  1.0   4  4.0  0.0  0.0  3.0  0.0   0.0  
schilcu012006  0.0  0.0   0  1.0  0.0  0.0  0.0  0.0   0.0  
myersmi012006  0.0  0.0   0  0.0  0.0  0.0  0.0  0.0   0.0  
helliri012006  0.0  0.0   0  2.0  0.0  0.0  0.0  0.0   0.0  
johnsra052006  0.0  0.0   0  4.0  0.0  0.0  0.0  0.0   0.0  

[5 rows x 22 columns]

This looks okay, but let's check:

In [51]:
baseball_newind.index.is_unique

False

So, indices need not be unique. Our choice is not unique because some players change teams within years.

In [52]:
pd.Series(baseball_newind.index).value_counts()

wickmbo012007    2
gomezch022007    2
sweenma012007    2
claytro012007    2
hernaro012007    2
loftoke012007    2
trachst012007    2
wellsda012007    2
francju012007    2
benitar012007    2
coninje012007    2
cirilje012007    2
piazzmi012007    1
griffke022007    1
stantmi022007    1
finlest012006    1
sprinru012007    1
guarded012007    1
thomeji012007    1
maddugr012007    1
biggicr012007    1
oliveda022007    1
vizquom012007    1
johnsra052007    1
wakefti012007    1
jonesto022007    1
aloumo012007     1
zaungr012007     1
womacto012006    1
finlest012007    1
                ..
smoltjo012007    1
easleda012007    1
ramirma022007    1
tavarju012007    1
loaizes012007    1
mabryjo012007    1
rogerke012007    1
delgaca012007    1
martipe022007    1
gonzalu012006    1
seleaa012007     1
valenjo032007    1
suppaje012007    1
stairma012007    1
johnsra052006    1
helliri012006    1
graffto012007    1
clemero022007    1
bondsba012007    1
weathda012007    1
myersmi012006    1
schilcu01200

The most important consequence of a non-unique index is that indexing by label will return multiple values for some labels:

In [53]:
baseball_newind.loc['wickmbo012007']

player  year  stint team  lg   g  ab  r  h  X2b  ...   rbi  \
wickmbo012007  wickmbo01  2007      2  ARI  NL   8   0  0  0    0  ...   0.0   
wickmbo012007  wickmbo01  2007      1  ATL  NL  47   0  0  0    0  ...   0.0   

                sb   cs  bb   so  ibb  hbp   sh   sf  gidp  
wickmbo012007  0.0  0.0   0  0.0  0.0  0.0  0.0  0.0   0.0  
wickmbo012007  0.0  0.0   0  0.0  0.0  0.0  0.0  0.0   0.0  

[2 rows x 22 columns]

We will learn more about indexing below.

We can create a truly unique index by combining `player`, `team` and `year`:

In [54]:
player_unique = baseball.player + baseball.team + baseball.year.astype(str)
baseball_newind = baseball.copy()
baseball_newind.index = player_unique
baseball_newind.head()

player  year  stint team  lg   g  ab  r   h  X2b  ...   \
womacto01CHN2006  womacto01  2006      2  CHN  NL  19  50  6  14    1  ...    
schilcu01BOS2006  schilcu01  2006      1  BOS  AL  31   2  0   1    0  ...    
myersmi01NYA2006  myersmi01  2006      1  NYA  AL  62   0  0   0    0  ...    
helliri01MIL2006  helliri01  2006      1  MIL  NL  20   3  0   0    0  ...    
johnsra05NYA2006  johnsra05  2006      1  NYA  AL  33   6  0   1    0  ...    

                  rbi   sb   cs  bb   so  ibb  hbp   sh   sf  gidp  
womacto01CHN2006  2.0  1.0  1.0   4  4.0  0.0  0.0  3.0  0.0   0.0  
schilcu01BOS2006  0.0  0.0  0.0   0  1.0  0.0  0.0  0.0  0.0   0.0  
myersmi01NYA2006  0.0  0.0  0.0   0  0.0  0.0  0.0  0.0  0.0   0.0  
helliri01MIL2006  0.0  0.0  0.0   0  2.0  0.0  0.0  0.0  0.0   0.0  
johnsra05NYA2006  0.0  0.0  0.0   0  4.0  0.0  0.0  0.0  0.0   0.0  

[5 rows x 22 columns]

In [55]:
baseball_newind.index.is_unique

True

We can create meaningful indices more easily using a hierarchical index; for now, we will stick with the numeric `id` field as our index.

### Manipulating indices

**Reindexing** allows users to manipulate the data labels in a DataFrame. It forces a DataFrame to conform to the new index, and optionally, fill in missing data if requested.

A simple use of `reindex` is to alter the order of the rows:

In [56]:
baseball.reindex(baseball.index[::-1]).head()

player  year  stint team  lg    g   ab   r    h  X2b  ...    rbi  \
id                                                              ...          
89534  alomasa02  2007      1  NYN  NL    8   22   1    3    1  ...    0.0   
89533   aloumo01  2007      1  NYN  NL   87  328  51  112   19  ...   49.0   
89530  ausmubr01  2007      1  HOU  NL  117  349  38   82   16  ...   25.0   
89526  benitar01  2007      1  SFN  NL   19    0   0    0    0  ...    0.0   
89525  benitar01  2007      2  FLO  NL   34    0   0    0    0  ...    0.0   

        sb   cs  bb    so  ibb  hbp   sh   sf  gidp  
id                                                   
89534  0.0  0.0   0   3.0  0.0  0.0  0.0  0.0   0.0  
89533  3.0  0.0  27  30.0  5.0  2.0  0.0  3.0  13.0  
89530  6.0  1.0  37  74.0  3.0  6.0  4.0  1.0  11.0  
89526  0.0  0.0   0   0.0  0.0  0.0  0.0  0.0   0.0  
89525  0.0  0.0   0   0.0  0.0  0.0  0.0  0.0   0.0  

[5 rows x 22 columns]

Notice that the `id` index is not sequential. Say we wanted to populate the table with every `id` value. We could specify and index that is a sequence from the first to the last `id` numbers in the database, and Pandas would fill in the missing data with `NaN` values:

In [57]:
id_range = range(baseball.index.values.min(), baseball.index.values.max())
baseball.reindex(id_range).head()

player    year  stint team   lg     g    ab    r     h  X2b  ...   \
id                                                                     ...    
88641  womacto01  2006.0    2.0  CHN   NL  19.0  50.0  6.0  14.0  1.0  ...    
88642        NaN     NaN    NaN  NaN  NaN   NaN   NaN  NaN   NaN  NaN  ...    
88643  schilcu01  2006.0    1.0  BOS   AL  31.0   2.0  0.0   1.0  0.0  ...    
88644        NaN     NaN    NaN  NaN  NaN   NaN   NaN  NaN   NaN  NaN  ...    
88645  myersmi01  2006.0    1.0  NYA   AL  62.0   0.0  0.0   0.0  0.0  ...    

       rbi   sb   cs   bb   so  ibb  hbp   sh   sf  gidp  
id                                                        
88641  2.0  1.0  1.0  4.0  4.0  0.0  0.0  3.0  0.0   0.0  
88642  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN  
88643  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0  
88644  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN  
88645  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  

[5 rows x 22 columns]

Missing values can be filled as desired, either with selected values, or by rule:

In [62]:
baseball.reindex(id_range, method='ffill').head()

player  year  stint team  lg   g  ab  r   h  X2b  ...   rbi   sb  \
id                                                          ...              
88641  womacto01  2006      2  CHN  NL  19  50  6  14    1  ...   2.0  1.0   
88642  womacto01  2006      2  CHN  NL  19  50  6  14    1  ...   2.0  1.0   
88643  schilcu01  2006      1  BOS  AL  31   2  0   1    0  ...   0.0  0.0   
88644  schilcu01  2006      1  BOS  AL  31   2  0   1    0  ...   0.0  0.0   
88645  myersmi01  2006      1  NYA  AL  62   0  0   0    0  ...   0.0  0.0   

        cs  bb   so  ibb  hbp   sh   sf  gidp  
id                                             
88641  1.0   4  4.0  0.0  0.0  3.0  0.0   0.0  
88642  1.0   4  4.0  0.0  0.0  3.0  0.0   0.0  
88643  0.0   0  1.0  0.0  0.0  0.0  0.0   0.0  
88644  0.0   0  1.0  0.0  0.0  0.0  0.0   0.0  
88645  0.0   0  0.0  0.0  0.0  0.0  0.0   0.0  

[5 rows x 22 columns]

In [59]:
baseball.reindex(id_range, fill_value='charliebrown', columns=['player']).head()

player
id                 
88641     womacto01
88642  charliebrown
88643     schilcu01
88644  charliebrown
88645     myersmi01

Keep in mind that `reindex` does not work if we pass a non-unique index series.

We can remove rows or columns via the `drop` method:

In [63]:
baseball.shape

(100, 22)

In [64]:
baseball.drop([89525, 89526])

player  year  stint team  lg    g   ab   r    h  X2b  ...    rbi  \
id                                                              ...          
88641  womacto01  2006      2  CHN  NL   19   50   6   14    1  ...    2.0   
88643  schilcu01  2006      1  BOS  AL   31    2   0    1    0  ...    0.0   
88645  myersmi01  2006      1  NYA  AL   62    0   0    0    0  ...    0.0   
88649  helliri01  2006      1  MIL  NL   20    3   0    0    0  ...    0.0   
88650  johnsra05  2006      1  NYA  AL   33    6   0    1    0  ...    0.0   
88652  finlest01  2006      1  SFN  NL  139  426  66  105   21  ...   40.0   
88653  gonzalu01  2006      1  ARI  NL  153  586  93  159   52  ...   73.0   
88662   seleaa01  2006      1  LAN  NL   28   26   2    5    1  ...    0.0   
89177  francju01  2007      2  ATL  NL   15   40   1   10    3  ...    8.0   
89178  francju01  2007      1  NYN  NL   40   50   7   10    0  ...    8.0   
89330   zaungr01  2007      1  TOR  AL  110  331  43   80   24  ...   52.0   
89333  witasja01  2007      1  TBA  AL    3    0   0    0    0  ...    0.0   
89334  williwo02  2007      1  HOU  NL   33   59   3    6    0  ...    2.0   
89335  wickmbo01  2007      2  ARI  NL    8    0   0    0    0  ...    0.0   
89336  wickmbo01  2007      1  ATL  NL   47    0   0    0    0  ...    0.0   
89337  whitero02  2007      1  MIN  AL   38  109   8   19    4  ...   20.0   
89338  whiteri01  2007      1  HOU  NL   20    1   0    0    0  ...    0.0   
89339  wellsda01  2007      2  LAN  NL    7   15   2    4    1  ...    1.0   
89340  wellsda01  2007      1  SDN  NL   22   38   1    4    0  ...    0.0   
89341  weathda01  2007      1  CIN  NL   67    0   0    0    0  ...    0.0   
89343  walketo04  2007      1  OAK  AL   18   48   5   13    1  ...    4.0   
89345  wakefti01  2007      1  BOS  AL    1    2   0    0    0  ...    0.0   
89347  vizquom01  2007      1  SFN  NL  145  513  54  126   18  ...   51.0   
89348  villoro01  2007      1  NYA  AL    6    0   0    0    0  ...    0.0   
89352  valenjo03  2007      1  NYN  NL   51  166  18   40   11  ...   18.0   
89354  trachst01  2007      2  CHN  NL    4    7   0    1    0  ...    0.0   
89355  trachst01  2007      1  BAL  AL    3    5   0    0    0  ...    0.0   
89359  timlimi01  2007      1  BOS  AL    4    0   0    0    0  ...    0.0   
89360  thomeji01  2007      1  CHA  AL  130  432  79  119   19  ...   96.0   
89361  thomafr04  2007      1  TOR  AL  155  531  63  147   30  ...   95.0   
...          ...   ...    ...  ...  ..  ...  ...  ..  ...  ...  ...    ...   
89451  hernaro01  2007      2  LAN  NL   22    0   0    0    0  ...    0.0   
89452  hernaro01  2007      1  CLE  AL    2    0   0    0    0  ...    0.0   
89460  guarded01  2007      1  CIN  NL   15    0   0    0    0  ...    0.0   
89462  griffke02  2007      1  CIN  NL  144  528  78  146   24  ...   93.0   
89463  greensh01  2007      1  NYN  NL  130  446  62  130   30  ...   46.0   
89464  graffto01  2007      1  MIL  NL   86  231  34   55    8  ...   30.0   
89465  gordoto01  2007      1  PHI  NL   44    0   0    0    0  ...    0.0   
89466  gonzalu01  2007      1  LAN  NL  139  464  70  129   23  ...   68.0   
89467  gomezch02  2007      2  CLE  AL   19   53   4   15    2  ...    5.0   
89468  gomezch02  2007      1  BAL  AL   73  169  17   51   10  ...   16.0   
89469  glavito02  2007      1  NYN  NL   33   56   3   12    1  ...    4.0   
89473  floydcl01  2007      1  CHN  NL  108  282  40   80   10  ...   45.0   
89474  finlest01  2007      1  COL  NL   43   94   9   17    3  ...    2.0   
89480  embreal01  2007      1  OAK  AL    4    0   0    0    0  ...    0.0   
89481  edmonji01  2007      1  SLN  NL  117  365  39   92   15  ...   53.0   
89482  easleda01  2007      1  NYN  NL   76  193  24   54    6  ...   26.0   
89489  delgaca01  2007      1  NYN  NL  139  538  71  139   30  ...   87.0   
89493  cormirh01  2007      1  CIN  NL    6    0   0    0    0  ...    0.0   
89494  coninje01  2007      2  N

In [65]:
baseball.drop(['ibb','hbp'], axis=1)

player  year  stint team  lg    g   ab   r    h  X2b  X3b  hr   rbi  \
id                                                                              
88641  womacto01  2006      2  CHN  NL   19   50   6   14    1    0   1   2.0   
88643  schilcu01  2006      1  BOS  AL   31    2   0    1    0    0   0   0.0   
88645  myersmi01  2006      1  NYA  AL   62    0   0    0    0    0   0   0.0   
88649  helliri01  2006      1  MIL  NL   20    3   0    0    0    0   0   0.0   
88650  johnsra05  2006      1  NYA  AL   33    6   0    1    0    0   0   0.0   
88652  finlest01  2006      1  SFN  NL  139  426  66  105   21   12   6  40.0   
88653  gonzalu01  2006      1  ARI  NL  153  586  93  159   52    2  15  73.0   
88662   seleaa01  2006      1  LAN  NL   28   26   2    5    1    0   0   0.0   
89177  francju01  2007      2  ATL  NL   15   40   1   10    3    0   0   8.0   
89178  francju01  2007      1  NYN  NL   40   50   7   10    0    0   1   8.0   
89330   zaungr01  2007      1  TOR  AL  110  331  43   80   24    1  10  52.0   
89333  witasja01  2007      1  TBA  AL    3    0   0    0    0    0   0   0.0   
89334  williwo02  2007      1  HOU  NL   33   59   3    6    0    0   1   2.0   
89335  wickmbo01  2007      2  ARI  NL    8    0   0    0    0    0   0   0.0   
89336  wickmbo01  2007      1  ATL  NL   47    0   0    0    0    0   0   0.0   
89337  whitero02  2007      1  MIN  AL   38  109   8   19    4    0   4  20.0   
89338  whiteri01  2007      1  HOU  NL   20    1   0    0    0    0   0   0.0   
89339  wellsda01  2007      2  LAN  NL    7   15   2    4    1    0   0   1.0   
89340  wellsda01  2007      1  SDN  NL   22   38   1    4    0    0   0   0.0   
89341  weathda01  2007      1  CIN  NL   67    0   0    0    0    0   0   0.0   
89343  walketo04  2007      1  OAK  AL   18   48   5   13    1    0   0   4.0   
89345  wakefti01  2007      1  BOS  AL    1    2   0    0    0    0   0   0.0   
89347  vizquom01  2007      1  SFN  NL  145  513  54  126   18    3   4  51.0   
89348  villoro01  2007      1  NYA  AL    6    0   0    0    0    0   0   0.0   
89352  valenjo03  2007      1  NYN  NL   51  166  18   40   11    1   3  18.0   
89354  trachst01  2007      2  CHN  NL    4    7   0    1    0    0   0   0.0   
89355  trachst01  2007      1  BAL  AL    3    5   0    0    0    0   0   0.0   
89359  timlimi01  2007      1  BOS  AL    4    0   0    0    0    0   0   0.0   
89360  thomeji01  2007      1  CHA  AL  130  432  79  119   19    0  35  96.0   
89361  thomafr04  2007      1  TOR  AL  155  531  63  147   30    0  26  95.0   
...          ...   ...    ...  ...  ..  ...  ...  ..  ...  ...  ...  ..   ...   
89460  guarded01  2007      1  CIN  NL   15    0   0    0    0    0   0   0.0   
89462  griffke02  2007      1  CIN  NL  144  528  78  146   24    1  30  93.0   
89463  greensh01  2007      1  NYN  NL  130  446  62  130   30    1  10  46.0   
89464  graffto01  2007      1  MIL  NL   86  231  34   55    8    0   9  30.0   
89465  gordoto01  2007      1  PHI  NL   44    0   0    0    0    0   0   0.0   
89466  gonzalu01  2007      1  LAN  NL  139  464  70  129   23    2  15  68.0   
89467  gomezch02  2007      2  CLE  AL   19   53   4   15    2    0   0   5.0   
89468  gomezch02  2007      1  BAL  AL   73  169  17   51   10    1   1  16.0   
89469  glavito02  2007      1  NYN  NL   33   56   3   12    1    0   0   4.0   
89473  floydcl01  2007      1  CHN  NL  108  282  40   80   10    1   9  45.0   
89474  finlest01  2007      1  COL  NL   43   94   9   17    3    0   1   2.0   
89480  embreal01  2007      1  OAK  AL    4    0   0    0    0    0   0   0.0   
89481  edmonji01  2007      1  SLN  NL  117  365  39   92   15    2  12  53.0   
89482  easleda01  2007      1  NYN  NL   76  193  24   54    6    0  10  26.0   
89489  delgaca01  2007      1  NYN  NL  139  538  71  139   30    0  24  87.0   
89493  cormirh01  2007      1  CIN  NL    6    0   0    0    0    0   0   0.0   
89494  coninje01  2007      2  NYN  NL   

## Indexing and Selection

Indexing works analogously to indexing in NumPy arrays, except we can use the labels in the `Index` object to extract values in addition to arrays of integers.

In [66]:
# Sample Series object
hits = baseball_newind.h
hits

womacto01CHN2006     14
schilcu01BOS2006      1
myersmi01NYA2006      0
helliri01MIL2006      0
johnsra05NYA2006      1
finlest01SFN2006    105
gonzalu01ARI2006    159
seleaa01LAN2006       5
francju01ATL2007     10
francju01NYN2007     10
zaungr01TOR2007      80
witasja01TBA2007      0
williwo02HOU2007      6
wickmbo01ARI2007      0
wickmbo01ATL2007      0
whitero02MIN2007     19
whiteri01HOU2007      0
wellsda01LAN2007      4
wellsda01SDN2007      4
weathda01CIN2007      0
walketo04OAK2007     13
wakefti01BOS2007      0
vizquom01SFN2007    126
villoro01NYA2007      0
valenjo03NYN2007     40
trachst01CHN2007      1
trachst01BAL2007      0
timlimi01BOS2007      0
thomeji01CHA2007    119
thomafr04TOR2007    147
                   ... 
guarded01CIN2007      0
griffke02CIN2007    146
greensh01NYN2007    130
graffto01MIL2007     55
gordoto01PHI2007      0
gonzalu01LAN2007    129
gomezch02CLE2007     15
gomezch02BAL2007     51
glavito02NYN2007     12
floydcl01CHN2007     80
finlest01COL2007

In [67]:
# Numpy-style indexing
hits[:3]

womacto01CHN2006    14
schilcu01BOS2006     1
myersmi01NYA2006     0
Name: h, dtype: int64

In [68]:
# Indexing by label
hits[['womacto01CHN2006','schilcu01BOS2006']]

womacto01CHN2006    14
schilcu01BOS2006     1
Name: h, dtype: int64

We can also slice with data labels, since they have an intrinsic order within the Index:

In [69]:
hits['womacto01CHN2006':'gonzalu01ARI2006']

womacto01CHN2006     14
schilcu01BOS2006      1
myersmi01NYA2006      0
helliri01MIL2006      0
johnsra05NYA2006      1
finlest01SFN2006    105
gonzalu01ARI2006    159
Name: h, dtype: int64

In [70]:
hits['womacto01CHN2006':'gonzalu01ARI2006'] = 5
hits

c:\python27\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


womacto01CHN2006      5
schilcu01BOS2006      5
myersmi01NYA2006      5
helliri01MIL2006      5
johnsra05NYA2006      5
finlest01SFN2006      5
gonzalu01ARI2006      5
seleaa01LAN2006       5
francju01ATL2007     10
francju01NYN2007     10
zaungr01TOR2007      80
witasja01TBA2007      0
williwo02HOU2007      6
wickmbo01ARI2007      0
wickmbo01ATL2007      0
whitero02MIN2007     19
whiteri01HOU2007      0
wellsda01LAN2007      4
wellsda01SDN2007      4
weathda01CIN2007      0
walketo04OAK2007     13
wakefti01BOS2007      0
vizquom01SFN2007    126
villoro01NYA2007      0
valenjo03NYN2007     40
trachst01CHN2007      1
trachst01BAL2007      0
timlimi01BOS2007      0
thomeji01CHA2007    119
thomafr04TOR2007    147
                   ... 
guarded01CIN2007      0
griffke02CIN2007    146
greensh01NYN2007    130
graffto01MIL2007     55
gordoto01PHI2007      0
gonzalu01LAN2007    129
gomezch02CLE2007     15
gomezch02BAL2007     51
glavito02NYN2007     12
floydcl01CHN2007     80
finlest01COL2007

In a `DataFrame` we can slice along either or both axes:

In [71]:
baseball_newind[['h','ab']]

h   ab
womacto01CHN2006    5   50
schilcu01BOS2006    5    2
myersmi01NYA2006    5    0
helliri01MIL2006    5    3
johnsra05NYA2006    5    6
finlest01SFN2006    5  426
gonzalu01ARI2006    5  586
seleaa01LAN2006     5   26
francju01ATL2007   10   40
francju01NYN2007   10   50
zaungr01TOR2007    80  331
witasja01TBA2007    0    0
williwo02HOU2007    6   59
wickmbo01ARI2007    0    0
wickmbo01ATL2007    0    0
whitero02MIN2007   19  109
whiteri01HOU2007    0    1
wellsda01LAN2007    4   15
wellsda01SDN2007    4   38
weathda01CIN2007    0    0
walketo04OAK2007   13   48
wakefti01BOS2007    0    2
vizquom01SFN2007  126  513
villoro01NYA2007    0    0
valenjo03NYN2007   40  166
trachst01CHN2007    1    7
trachst01BAL2007    0    5
timlimi01BOS2007    0    0
thomeji01CHA2007  119  432
thomafr04TOR2007  147  531
...               ...  ...
guarded01CIN2007    0    0
griffke02CIN2007  146  528
greensh01NYN2007  130  446
graffto01MIL2007   55  231
gordoto01PHI2007    0    0
gonzalu01LAN2007  129  464
gomezch02CLE2007   15   53
gomezch02BAL2007   51  169
glavito02NYN2007   12   56
floydcl01CHN2007   80  282
finlest01COL2007   17   94
embreal01OAK2007    0    0
edmonji01SLN2007   92  365
easleda01NYN2007   54  193
delgaca01NYN2007  139  538
cormirh01CIN2007    0    0
coninje01NYN2007    8   41
coninje01CIN2007   57  215
clemero02NYA2007    1    2
claytro01BOS2007    0    6
claytro01TOR2007   48  189
cirilje01ARI2007    8   40
cirilje01MIN2007   40  153
bondsba01SFN2007   94  340
biggicr01HOU2007  130  517
benitar01FLO2007    0    0
benitar01SFN2007    0    0
ausmubr01HOU2007   82  349
aloumo01NYN2007   112  328
alomasa02NYN2007    3   22

[100 rows x 2 columns]

In [72]:
baseball_newind[baseball_newind.ab>500]

player  year  stint team  lg    g   ab   r    h  X2b  \
gonzalu01ARI2006  gonzalu01  2006      1  ARI  NL  153  586  93    5   52   
vizquom01SFN2007  vizquom01  2007      1  SFN  NL  145  513  54  126   18   
thomafr04TOR2007  thomafr04  2007      1  TOR  AL  155  531  63  147   30   
rodriiv01DET2007  rodriiv01  2007      1  DET  AL  129  502  50  141   31   
griffke02CIN2007  griffke02  2007      1  CIN  NL  144  528  78  146   24   
delgaca01NYN2007  delgaca01  2007      1  NYN  NL  139  538  71  139   30   
biggicr01HOU2007  biggicr01  2007      1  HOU  NL  141  517  68  130   31   

                  ...    rbi    sb   cs  bb     so   ibb   hbp    sh   sf  \
gonzalu01ARI2006  ...   73.0   0.0  1.0  69   58.0  10.0   7.0   0.0  6.0   
vizquom01SFN2007  ...   51.0  14.0  6.0  44   48.0   6.0   1.0  14.0  3.0   
thomafr04TOR2007  ...   95.0   0.0  0.0  81   94.0   3.0   7.0   0.0  5.0   
rodriiv01DET2007  ...   63.0   2.0  2.0   9   96.0   1.0   1.0   1.0  2.0   
griffke02CIN2007  ...   93.0   6.0  1.0  85   99.0  14.0   1.0   0.0  9.0   
delgaca01NYN2007  ...   87.0   4.0  0.0  52  118.0   8.0  11.0   0.0  6.0   
biggicr01HOU2007  ...   50.0   4.0  3.0  23  112.0   0.0   3.0   7.0  5.0   

                  gidp  
gonzalu01ARI2006  14.0  
vizquom01SFN2007  14.0  
thomafr04TOR2007  14.0  
rodriiv01DET2007  16.0  
griffke02CIN2007  14.0  
delgaca01NYN2007  12.0  
biggicr01HOU2007   5.0  

[7 rows x 22 columns]

For a more concise (and readable) syntax, we can use the new `query` method to perform selection on a `DataFrame`. Instead of having to type the fully-specified column, we can simply pass a string that describes what to select. The query above is then simply:

In [73]:
baseball_newind.query('ab > 500')

player  year  stint team  lg    g   ab   r    h  X2b  \
gonzalu01ARI2006  gonzalu01  2006      1  ARI  NL  153  586  93    5   52   
vizquom01SFN2007  vizquom01  2007      1  SFN  NL  145  513  54  126   18   
thomafr04TOR2007  thomafr04  2007      1  TOR  AL  155  531  63  147   30   
rodriiv01DET2007  rodriiv01  2007      1  DET  AL  129  502  50  141   31   
griffke02CIN2007  griffke02  2007      1  CIN  NL  144  528  78  146   24   
delgaca01NYN2007  delgaca01  2007      1  NYN  NL  139  538  71  139   30   
biggicr01HOU2007  biggicr01  2007      1  HOU  NL  141  517  68  130   31   

                  ...    rbi    sb   cs  bb     so   ibb   hbp    sh   sf  \
gonzalu01ARI2006  ...   73.0   0.0  1.0  69   58.0  10.0   7.0   0.0  6.0   
vizquom01SFN2007  ...   51.0  14.0  6.0  44   48.0   6.0   1.0  14.0  3.0   
thomafr04TOR2007  ...   95.0   0.0  0.0  81   94.0   3.0   7.0   0.0  5.0   
rodriiv01DET2007  ...   63.0   2.0  2.0   9   96.0   1.0   1.0   1.0  2.0   
griffke02CIN2007  ...   93.0   6.0  1.0  85   99.0  14.0   1.0   0.0  9.0   
delgaca01NYN2007  ...   87.0   4.0  0.0  52  118.0   8.0  11.0   0.0  6.0   
biggicr01HOU2007  ...   50.0   4.0  3.0  23  112.0   0.0   3.0   7.0  5.0   

                  gidp  
gonzalu01ARI2006  14.0  
vizquom01SFN2007  14.0  
thomafr04TOR2007  14.0  
rodriiv01DET2007  16.0  
griffke02CIN2007  14.0  
delgaca01NYN2007  12.0  
biggicr01HOU2007   5.0  

[7 rows x 22 columns]

The `DataFrame.index` and `DataFrame.columns` are placed in the query namespace by default. If you want to refer to a variable in the current namespace, you can prefix the variable with `@`:

In [74]:
min_ab = 450

In [75]:
baseball_newind.query('ab > @min_ab')

player  year  stint team  lg    g   ab    r    h  X2b  \
gonzalu01ARI2006  gonzalu01  2006      1  ARI  NL  153  586   93    5   52   
vizquom01SFN2007  vizquom01  2007      1  SFN  NL  145  513   54  126   18   
thomafr04TOR2007  thomafr04  2007      1  TOR  AL  155  531   63  147   30   
sheffga01DET2007  sheffga01  2007      1  DET  AL  133  494  107  131   20   
rodriiv01DET2007  rodriiv01  2007      1  DET  AL  129  502   50  141   31   
ramirma02BOS2007  ramirma02  2007      1  BOS  AL  133  483   84  143   33   
kentje01LAN2007    kentje01  2007      1  LAN  NL  136  494   78  149   36   
griffke02CIN2007  griffke02  2007      1  CIN  NL  144  528   78  146   24   
gonzalu01LAN2007  gonzalu01  2007      1  LAN  NL  139  464   70  129   23   
delgaca01NYN2007  delgaca01  2007      1  NYN  NL  139  538   71  139   30   
biggicr01HOU2007  biggicr01  2007      1  HOU  NL  141  517   68  130   31   

                  ...    rbi    sb   cs  bb     so   ibb   hbp    sh   sf  \
gonzalu01ARI2006  ...   73.0   0.0  1.0  69   58.0  10.0   7.0   0.0  6.0   
vizquom01SFN2007  ...   51.0  14.0  6.0  44   48.0   6.0   1.0  14.0  3.0   
thomafr04TOR2007  ...   95.0   0.0  0.0  81   94.0   3.0   7.0   0.0  5.0   
sheffga01DET2007  ...   75.0  22.0  5.0  84   71.0   2.0   9.0   0.0  6.0   
rodriiv01DET2007  ...   63.0   2.0  2.0   9   96.0   1.0   1.0   1.0  2.0   
ramirma02BOS2007  ...   88.0   0.0  0.0  71   92.0  13.0   7.0   0.0  8.0   
kentje01LAN2007   ...   79.0   1.0  3.0  57   61.0   4.0   5.0   0.0  6.0   
griffke02CIN2007  ...   93.0   6.0  1.0  85   99.0  14.0   1.0   0.0  9.0   
gonzalu01LAN2007  ...   68.0   6.0  2.0  56   56.0   4.0   4.0   0.0  2.0   
delgaca01NYN2007  ...   87.0   4.0  0.0  52  118.0   8.0  11.0   0.0  6.0   
biggicr01HOU2007  ...   50.0   4.0  3.0  23  112.0   0.0   3.0   7.0  5.0   

                  gidp  
gonzalu01ARI2006  14.0  
vizquom01SFN2007  14.0  
thomafr04TOR2007  14.0  
sheffga01DET2007  10.0  
rodriiv01DET2007  16.0  
ramirma02BOS2007  21.0  
kentje01LAN2007   17.0  
griffke02CIN2007  14.0  
gonzalu01LAN2007  11.0  
delgaca01NYN2007  12.0  
biggicr01HOU2007   5.0  

[11 rows x 22 columns]

The indexing field `loc` allows us to select subsets of rows and columns in an intuitive way:

In [76]:
baseball_newind.loc['gonzalu01ARI2006', ['h','X2b', 'X3b', 'hr']]

h       5
X2b    52
X3b     2
hr     15
Name: gonzalu01ARI2006, dtype: object

In [77]:
baseball_newind.loc[:'myersmi01NYA2006', 'hr']

womacto01CHN2006    1
schilcu01BOS2006    0
myersmi01NYA2006    0
Name: hr, dtype: int64

In addition to using `loc` to select rows and columns by **label**, pandas also allows indexing by **position** using the `iloc` attribute.

So, we can query rows and columns by absolute position, rather than by name:

In [78]:
baseball_newind.iloc[:5, 5:8]

g  ab  r
womacto01CHN2006  19  50  6
schilcu01BOS2006  31   2  0
myersmi01NYA2006  62   0  0
helliri01MIL2006  20   3  0
johnsra05NYA2006  33   6  0

### Exercise

You can use the `isin` method query a DataFrame based upon a list of values as follows: 

    data['phylum'].isin(['Firmacutes', 'Bacteroidetes'])

Use `isin` to find all players that played for the Los Angeles Dodgers (LAN) or the San Francisco Giants (SFN). How many records contain these values?

In [79]:
# Write your answer here

## Operations

`DataFrame` and `Series` objects allow for several operations to take place either on a single object, or between two or more objects.

For example, we can perform arithmetic on the elements of two objects, such as combining baseball statistics across years. First, let's (artificially) construct two Series, consisting of home runs hit in years 2006 and 2007, respectively:

In [80]:
hr2006 = baseball.loc[baseball.year==2006, 'hr']
hr2006.index = baseball.player[baseball.year==2006]

hr2007 = baseball.loc[baseball.year==2007, 'hr']
hr2007.index = baseball.player[baseball.year==2007]

In [81]:
hr2007

player
francju01     0
francju01     1
zaungr01     10
witasja01     0
williwo02     1
wickmbo01     0
wickmbo01     0
whitero02     4
whiteri01     0
wellsda01     0
wellsda01     0
weathda01     0
walketo04     0
wakefti01     0
vizquom01     4
villoro01     0
valenjo03     3
trachst01     0
trachst01     0
timlimi01     0
thomeji01    35
thomafr04    26
tavarju01     0
sweenma01     0
sweenma01     2
suppaje01     0
stinnke01     1
stantmi02     0
stairma01    21
sprinru01     0
             ..
guarded01     0
griffke02    30
greensh01    10
graffto01     9
gordoto01     0
gonzalu01    15
gomezch02     0
gomezch02     1
glavito02     0
floydcl01     9
finlest01     1
embreal01     0
edmonji01    12
easleda01    10
delgaca01    24
cormirh01     0
coninje01     0
coninje01     6
clemero02     0
claytro01     0
claytro01     1
cirilje01     0
cirilje01     2
bondsba01    28
biggicr01    10
benitar01     0
benitar01     0
ausmubr01     3
aloumo01     13
alomasa02     0
Name: hr, Length:

Now, let's add them together, in hopes of getting 2-year home run totals:

In [82]:
hr_total = hr2006 + hr2007
hr_total

player
alomasa02     NaN
aloumo01      NaN
ausmubr01     NaN
benitar01     NaN
benitar01     NaN
biggicr01     NaN
bondsba01     NaN
cirilje01     NaN
cirilje01     NaN
claytro01     NaN
claytro01     NaN
clemero02     NaN
coninje01     NaN
coninje01     NaN
cormirh01     NaN
delgaca01     NaN
easleda01     NaN
edmonji01     NaN
embreal01     NaN
finlest01     7.0
floydcl01     NaN
francju01     NaN
francju01     NaN
glavito02     NaN
gomezch02     NaN
gomezch02     NaN
gonzalu01    30.0
gordoto01     NaN
graffto01     NaN
greensh01     NaN
             ... 
sosasa01      NaN
sprinru01     NaN
stairma01     NaN
stantmi02     NaN
stinnke01     NaN
suppaje01     NaN
sweenma01     NaN
sweenma01     NaN
tavarju01     NaN
thomafr04     NaN
thomeji01     NaN
timlimi01     NaN
trachst01     NaN
trachst01     NaN
valenjo03     NaN
villoro01     NaN
vizquom01     NaN
wakefti01     NaN
walketo04     NaN
weathda01     NaN
wellsda01     NaN
wellsda01     NaN
whiteri01     NaN
whitero02     NaN
wic

Pandas' data alignment places `NaN` values for labels that do not overlap in the two Series. In fact, there are only 6 players that occur in both years.

In [83]:
hr_total[hr_total.notnull()]

player
finlest01     7.0
gonzalu01    30.0
johnsra05     0.0
myersmi01     0.0
schilcu01     0.0
seleaa01      0.0
Name: hr, dtype: float64

While we do want the operation to honor the data labels in this way, we probably do not want the missing values to be filled with `NaN`. We can use the `add` method to calculate player home run totals by using the `fill_value` argument to insert a zero for home runs where labels do not overlap:

In [84]:
hr2007.add(hr2006, fill_value=0)

player
alomasa02     0.0
aloumo01     13.0
ausmubr01     3.0
benitar01     0.0
benitar01     0.0
biggicr01    10.0
bondsba01    28.0
cirilje01     0.0
cirilje01     2.0
claytro01     0.0
claytro01     1.0
clemero02     0.0
coninje01     0.0
coninje01     6.0
cormirh01     0.0
delgaca01    24.0
easleda01    10.0
edmonji01    12.0
embreal01     0.0
finlest01     7.0
floydcl01     9.0
francju01     0.0
francju01     1.0
glavito02     0.0
gomezch02     0.0
gomezch02     1.0
gonzalu01    30.0
gordoto01     0.0
graffto01     9.0
greensh01    10.0
             ... 
sosasa01     21.0
sprinru01     0.0
stairma01    21.0
stantmi02     0.0
stinnke01     1.0
suppaje01     0.0
sweenma01     0.0
sweenma01     2.0
tavarju01     0.0
thomafr04    26.0
thomeji01    35.0
timlimi01     0.0
trachst01     0.0
trachst01     0.0
valenjo03     3.0
villoro01     0.0
vizquom01     4.0
wakefti01     0.0
walketo04     0.0
weathda01     0.0
wellsda01     0.0
wellsda01     0.0
whiteri01     0.0
whitero02     4.0
wic

Operations can also be **broadcast** between rows or columns.

For example, if we subtract the maximum number of home runs hit from the `hr` column, we get how many fewer than the maximum were hit by each player:

In [85]:
baseball.hr - baseball.hr.max()

id
88641   -34
88643   -35
88645   -35
88649   -35
88650   -35
88652   -29
88653   -20
88662   -35
89177   -35
89178   -34
89330   -25
89333   -35
89334   -34
89335   -35
89336   -35
89337   -31
89338   -35
89339   -35
89340   -35
89341   -35
89343   -35
89345   -35
89347   -31
89348   -35
89352   -32
89354   -35
89355   -35
89359   -35
89360     0
89361    -9
         ..
89460   -35
89462    -5
89463   -25
89464   -26
89465   -35
89466   -20
89467   -35
89468   -34
89469   -35
89473   -26
89474   -34
89480   -35
89481   -23
89482   -25
89489   -11
89493   -35
89494   -35
89495   -29
89497   -35
89498   -35
89499   -34
89501   -35
89502   -33
89521    -7
89523   -25
89525   -35
89526   -35
89530   -32
89533   -22
89534   -35
Name: hr, Length: 100, dtype: int64

Or, looking at things row-wise, we can see how a particular player compares with the rest of the group with respect to important statistics

In [86]:
baseball.loc[89521, "player"]

'bondsba01'

In [87]:
stats = baseball[['h','X2b', 'X3b', 'hr']]
diff = stats - stats.loc[89521]
diff[:10]

h  X2b  X3b  hr
id                     
88641 -80  -13    0 -27
88643 -93  -14    0 -28
88645 -94  -14    0 -28
88649 -94  -14    0 -28
88650 -93  -14    0 -28
88652  11    7   12 -22
88653  65   38    2 -13
88662 -89  -13    0 -28
89177 -84  -11    0 -28
89178 -84  -14    0 -27

We can also apply functions to each column or row of a `DataFrame`

In [88]:
stats.apply(np.median)

h      8.0
X2b    1.0
X3b    0.0
hr     0.0
dtype: float64

In [89]:
def range_calc(x):
    return x.max() - x.min()

## Sorting and Ranking

Pandas objects include methods for re-ordering data.

In [90]:
baseball_newind.sort_index().head()

player  year  stint team  lg    g   ab   r    h  X2b  \
alomasa02NYN2007  alomasa02  2007      1  NYN  NL    8   22   1    3    1   
aloumo01NYN2007    aloumo01  2007      1  NYN  NL   87  328  51  112   19   
ausmubr01HOU2007  ausmubr01  2007      1  HOU  NL  117  349  38   82   16   
benitar01FLO2007  benitar01  2007      2  FLO  NL   34    0   0    0    0   
benitar01SFN2007  benitar01  2007      1  SFN  NL   19    0   0    0    0   

                  ...    rbi   sb   cs  bb    so  ibb  hbp   sh   sf  gidp  
alomasa02NYN2007  ...    0.0  0.0  0.0   0   3.0  0.0  0.0  0.0  0.0   0.0  
aloumo01NYN2007   ...   49.0  3.0  0.0  27  30.0  5.0  2.0  0.0  3.0  13.0  
ausmubr01HOU2007  ...   25.0  6.0  1.0  37  74.0  3.0  6.0  4.0  1.0  11.0  
benitar01FLO2007  ...    0.0  0.0  0.0   0   0.0  0.0  0.0  0.0  0.0   0.0  
benitar01SFN2007  ...    0.0  0.0  0.0   0   0.0  0.0  0.0  0.0  0.0   0.0  

[5 rows x 22 columns]

In [91]:
baseball_newind.sort_index(ascending=False).head()

player  year  stint team  lg    g   ab   r   h  X2b  \
zaungr01TOR2007    zaungr01  2007      1  TOR  AL  110  331  43  80   24   
womacto01CHN2006  womacto01  2006      2  CHN  NL   19   50   6   5    1   
witasja01TBA2007  witasja01  2007      1  TBA  AL    3    0   0   0    0   
williwo02HOU2007  williwo02  2007      1  HOU  NL   33   59   3   6    0   
wickmbo01ATL2007  wickmbo01  2007      1  ATL  NL   47    0   0   0    0   

                  ...    rbi   sb   cs  bb    so  ibb  hbp   sh   sf  gidp  
zaungr01TOR2007   ...   52.0  0.0  0.0  51  55.0  8.0  2.0  1.0  6.0   9.0  
womacto01CHN2006  ...    2.0  1.0  1.0   4   4.0  0.0  0.0  3.0  0.0   0.0  
witasja01TBA2007  ...    0.0  0.0  0.0   0   0.0  0.0  0.0  0.0  0.0   0.0  
williwo02HOU2007  ...    2.0  0.0  0.0   0  25.0  0.0  0.0  5.0  0.0   1.0  
wickmbo01ATL2007  ...    0.0  0.0  0.0   0   0.0  0.0  0.0  0.0  0.0   0.0  

[5 rows x 22 columns]

Try sorting the **columns** instead of the rows, in ascending order:

In [92]:
baseball_newind.sort_index(axis=1).head()

X2b  X3b  ab  bb   cs   g  gidp  h  hbp  hr  ...   \
womacto01CHN2006    1    0  50   4  1.0  19   0.0  5  0.0   1  ...    
schilcu01BOS2006    0    0   2   0  0.0  31   0.0  5  0.0   0  ...    
myersmi01NYA2006    0    0   0   0  0.0  62   0.0  5  0.0   0  ...    
helliri01MIL2006    0    0   3   0  0.0  20   0.0  5  0.0   0  ...    
johnsra05NYA2006    0    0   6   0  0.0  33   0.0  5  0.0   0  ...    

                     player  r  rbi   sb   sf   sh   so  stint  team  year  
womacto01CHN2006  womacto01  6  2.0  1.0  0.0  3.0  4.0      2   CHN  2006  
schilcu01BOS2006  schilcu01  0  0.0  0.0  0.0  0.0  1.0      1   BOS  2006  
myersmi01NYA2006  myersmi01  0  0.0  0.0  0.0  0.0  0.0      1   NYA  2006  
helliri01MIL2006  helliri01  0  0.0  0.0  0.0  0.0  2.0      1   MIL  2006  
johnsra05NYA2006  johnsra05  0  0.0  0.0  0.0  0.0  4.0      1   NYA  2006  

[5 rows x 22 columns]

We can also use `sort_values` to sort a `Series` by value, rather than by label.

In [93]:
baseball.hr.sort_values()

id
89534     0
89365     0
89469     0
89367     0
89370     0
89467     0
89372     0
89465     0
89375     0
89460     0
89381     0
89382     0
89452     0
89363     0
89384     0
89388     0
89450     0
89445     0
89442     0
89431     0
89402     0
89406     0
89410     0
89411     0
89412     0
89420     0
89421     0
89451     0
89425     0
89429     0
         ..
89530     3
89352     3
89337     4
89347     4
89438     6
89495     6
88652     6
89430     7
89398     8
89473     9
89464     9
89482    10
89463    10
89330    10
89523    10
89389    11
89481    12
89533    13
89466    15
88653    15
89439    20
89396    20
89374    21
89371    21
89489    24
89378    25
89361    26
89521    28
89462    30
89360    35
Name: hr, Length: 100, dtype: int64

For a `DataFrame`, we can sort according to the values of one or more columns using the `by` argument of `sort_values`:

In [94]:
baseball[['player','sb','cs']].sort_values(ascending=[False,True], 
                                           by=['sb', 'cs']).head(10)

player    sb   cs
id                         
89378  sheffga01  22.0  5.0
89430  loftoke01  21.0  4.0
89347  vizquom01  14.0  6.0
89463  greensh01  11.0  1.0
88652  finlest01   7.0  0.0
89462  griffke02   6.0  1.0
89530  ausmubr01   6.0  1.0
89466  gonzalu01   6.0  2.0
89521  bondsba01   5.0  0.0
89438  kleskry01   5.0  1.0

**Ranking** does not re-arrange data, but instead returns an index that ranks each value relative to others in the Series.

In [95]:
baseball.hr.rank()

id
88641     62.5
88643     29.0
88645     29.0
88649     29.0
88650     29.0
88652     76.0
88653     89.5
88662     29.0
89177     29.0
89178     62.5
89330     83.5
89333     29.0
89334     62.5
89335     29.0
89336     29.0
89337     73.5
89338     29.0
89339     29.0
89340     29.0
89341     29.0
89343     29.0
89345     29.0
89347     73.5
89348     29.0
89352     71.5
89354     29.0
89355     29.0
89359     29.0
89360    100.0
89361     97.0
         ...  
89460     29.0
89462     99.0
89463     83.5
89464     80.5
89465     29.0
89466     89.5
89467     29.0
89468     62.5
89469     29.0
89473     80.5
89474     62.5
89480     29.0
89481     87.0
89482     83.5
89489     95.0
89493     29.0
89494     29.0
89495     76.0
89497     29.0
89498     29.0
89499     62.5
89501     29.0
89502     69.0
89521     98.0
89523     83.5
89525     29.0
89526     29.0
89530     71.5
89533     88.0
89534     29.0
Name: hr, Length: 100, dtype: float64

Ties are assigned the mean value of the tied ranks, which may result in decimal values.

In [96]:
pd.Series([100,100]).rank()

0    1.5
1    1.5
dtype: float64

Alternatively, you can break ties via one of several methods, such as by the order in which they occur in the dataset:

In [97]:
baseball.hr.rank(method='first')

id
88641     58.0
88643      1.0
88645      2.0
88649      3.0
88650      4.0
88652     75.0
88653     89.0
88662      5.0
89177      6.0
89178     59.0
89330     82.0
89333      7.0
89334     60.0
89335      8.0
89336      9.0
89337     73.0
89338     10.0
89339     11.0
89340     12.0
89341     13.0
89343     14.0
89345     15.0
89347     74.0
89348     16.0
89352     71.0
89354     17.0
89355     18.0
89359     19.0
89360    100.0
89361     97.0
         ...  
89460     45.0
89462     99.0
89463     83.0
89464     80.0
89465     46.0
89466     90.0
89467     47.0
89468     65.0
89469     48.0
89473     81.0
89474     66.0
89480     49.0
89481     87.0
89482     84.0
89489     95.0
89493     50.0
89494     51.0
89495     77.0
89497     52.0
89498     53.0
89499     67.0
89501     54.0
89502     70.0
89521     98.0
89523     85.0
89525     55.0
89526     56.0
89530     72.0
89533     88.0
89534     57.0
Name: hr, Length: 100, dtype: float64

Calling the `DataFrame`'s `rank` method results in the ranks of all columns:

In [98]:
baseball.rank(ascending=False).head()

player  year  stint  team    lg     g    ab     r     h   X2b  ...   \
id                                                                    ...    
88641     2.0  96.5    7.0  82.0  31.5  70.0  47.5  40.5  39.0  50.5  ...    
88643    37.5  96.5   57.0  88.0  81.5  55.5  73.0  81.0  63.5  78.0  ...    
88645    47.5  96.5   57.0  40.5  81.5  36.0  91.0  81.0  84.5  78.0  ...    
88649    66.0  96.5   57.0  47.0  31.5  67.5  69.0  81.0  84.5  78.0  ...    
88650    61.5  96.5   57.0  40.5  81.5  51.0  64.5  81.0  63.5  78.0  ...    

        rbi    sb    cs    bb    so   ibb   hbp    sh    sf  gidp  
id                                                                 
88641  51.0  24.5  17.5  44.5  59.0  66.0  65.5  16.0  70.0  76.5  
88643  78.5  63.5  62.5  79.0  73.0  66.0  65.5  67.5  70.0  76.5  
88645  78.5  63.5  62.5  79.0  89.0  66.0  65.5  67.5  70.0  76.5  
88649  78.5  63.5  62.5  79.0  67.0  66.0  65.5  67.5  70.0  76.5  
88650  78.5  63.5  62.5  79.0  59.0  66.0  65.5  67.5  70.0  76.5  

[5 rows x 22 columns]

In [99]:
baseball[['r','h','hr']].rank(ascending=False).head()

r     h    hr
id                     
88641  40.5  39.0  38.5
88643  81.0  63.5  72.0
88645  81.0  84.5  72.0
88649  81.0  84.5  72.0
88650  81.0  63.5  72.0

### Exercise

Calculate **on base percentage** for each player, and return the ordered series of estimates.

$$OBP = \frac{H + BB + HBP}{AB + BB + HBP + SF}$$

In [100]:
# Write your answer here

## Missing data

The occurence of missing data is so prevalent that it pays to use tools like Pandas, which seamlessly integrates missing data handling so that it can be dealt with easily, and in the manner required by the analysis at hand.

Missing data are represented in `Series` and `DataFrame` objects by the `NaN` floating point value. However, `None` is also treated as missing, since it is commonly used as such in other contexts (*e.g.* NumPy).

In [101]:
foo = pd.Series([np.nan, -3, None, 'foobar'])
foo

0       NaN
1        -3
2      None
3    foobar
dtype: object

In [102]:
foo.isnull()

0     True
1    False
2     True
3    False
dtype: bool

Missing values may be dropped or indexed out:

In [103]:
bacteria2

phylum
Firmicutes           NaN
Proteobacteria     632.0
Actinobacteria    1638.0
Bacteroidetes      569.0
dtype: float64

In [104]:
bacteria2.dropna()

phylum
Proteobacteria     632.0
Actinobacteria    1638.0
Bacteroidetes      569.0
dtype: float64

In [105]:
bacteria2.isnull()

phylum
Firmicutes         True
Proteobacteria    False
Actinobacteria    False
Bacteroidetes     False
dtype: bool

In [106]:
bacteria2[bacteria2.notnull()]

phylum
Proteobacteria     632.0
Actinobacteria    1638.0
Bacteroidetes      569.0
dtype: float64

By default, `dropna` drops entire rows in which one or more values are missing.

In [107]:
data.dropna()

patient          phylum  value  year  treatment month
0        1      Firmicutes    632  2013        0.0   Jan
1        1  Proteobacteria   1638  2013        0.0   Jan
2        1  Actinobacteria    569  2013        0.0   Jan
3        1   Bacteroidetes     14  2013        0.0   Jan
4        2      Firmicutes     21  2013        1.0   Jan
5        2  Proteobacteria      0  2013        1.0   Jan

This can be overridden by passing the `how='all'` argument, which only drops a row when every field is a missing value.

In [108]:
data.dropna(how='all')

patient          phylum  value  year  treatment month
0        1      Firmicutes    632  2013        0.0   Jan
1        1  Proteobacteria   1638  2013        0.0   Jan
2        1  Actinobacteria    569  2013        0.0   Jan
3        1   Bacteroidetes     14  2013        0.0   Jan
4        2      Firmicutes     21  2013        1.0   Jan
5        2  Proteobacteria      0  2013        1.0   Jan
6        2  Actinobacteria      5  2013        NaN   Jan
7        2   Bacteroidetes    555  2013        NaN   Jan

This can be customized further by specifying how many values need to be present before a row is dropped via the `thresh` argument.

In [109]:
data.loc[7, 'year'] = np.nan
data

patient          phylum  value    year  treatment month
0        1      Firmicutes    632  2013.0        0.0   Jan
1        1  Proteobacteria   1638  2013.0        0.0   Jan
2        1  Actinobacteria    569  2013.0        0.0   Jan
3        1   Bacteroidetes     14  2013.0        0.0   Jan
4        2      Firmicutes     21  2013.0        1.0   Jan
5        2  Proteobacteria      0  2013.0        1.0   Jan
6        2  Actinobacteria      5  2013.0        NaN   Jan
7        2   Bacteroidetes    555     NaN        NaN   Jan

In [110]:
data.dropna(thresh=4)

patient          phylum  value    year  treatment month
0        1      Firmicutes    632  2013.0        0.0   Jan
1        1  Proteobacteria   1638  2013.0        0.0   Jan
2        1  Actinobacteria    569  2013.0        0.0   Jan
3        1   Bacteroidetes     14  2013.0        0.0   Jan
4        2      Firmicutes     21  2013.0        1.0   Jan
5        2  Proteobacteria      0  2013.0        1.0   Jan
6        2  Actinobacteria      5  2013.0        NaN   Jan
7        2   Bacteroidetes    555     NaN        NaN   Jan

This is typically used in time series applications, where there are repeated measurements that are incomplete for some subjects.

### Exercise

Try using the `axis` argument to drop columns with missing values:

In [111]:
# Write your answer here

Rather than omitting missing data from an analysis, in some cases it may be suitable to fill the missing value in, either with a default value (such as zero) or a value that is either imputed or carried forward/backward from similar data points. We can do this programmatically in Pandas with the `fillna` argument.

In [112]:
bacteria2.fillna(0)

phylum
Firmicutes           0.0
Proteobacteria     632.0
Actinobacteria    1638.0
Bacteroidetes      569.0
dtype: float64

In [113]:
data.fillna({'year': 2013, 'treatment':2})

patient          phylum  value    year  treatment month
0        1      Firmicutes    632  2013.0        0.0   Jan
1        1  Proteobacteria   1638  2013.0        0.0   Jan
2        1  Actinobacteria    569  2013.0        0.0   Jan
3        1   Bacteroidetes     14  2013.0        0.0   Jan
4        2      Firmicutes     21  2013.0        1.0   Jan
5        2  Proteobacteria      0  2013.0        1.0   Jan
6        2  Actinobacteria      5  2013.0        2.0   Jan
7        2   Bacteroidetes    555  2013.0        2.0   Jan

Notice that `fillna` by default returns a new object with the desired filling behavior, rather than changing the `Series` or  `DataFrame` in place (**in general, we like to do this, by the way!**).

We can alter values in-place using `inplace=True`.

In [114]:
data.year.fillna(2013, inplace=True)
data

patient          phylum  value    year  treatment month
0        1      Firmicutes    632  2013.0        0.0   Jan
1        1  Proteobacteria   1638  2013.0        0.0   Jan
2        1  Actinobacteria    569  2013.0        0.0   Jan
3        1   Bacteroidetes     14  2013.0        0.0   Jan
4        2      Firmicutes     21  2013.0        1.0   Jan
5        2  Proteobacteria      0  2013.0        1.0   Jan
6        2  Actinobacteria      5  2013.0        NaN   Jan
7        2   Bacteroidetes    555  2013.0        NaN   Jan

Missing values can also be interpolated, using any one of a variety of methods:

In [115]:
bacteria2.fillna(method='bfill')

phylum
Firmicutes         632.0
Proteobacteria     632.0
Actinobacteria    1638.0
Bacteroidetes      569.0
dtype: float64

## Data summarization

We often wish to summarize data in `Series` or `DataFrame` objects, so that they can more easily be understood or compared with similar data. The NumPy package contains several functions that are useful here, but several summarization or reduction methods are built into Pandas data structures.

In [116]:
baseball.sum()

player    womacto01schilcu01myersmi01helliri01johnsra05f...
year                                                 200692
stint                                                   113
team      CHNBOSNYAMILNYASFNARILANATLNYNTORTBAHOUARIATLM...
lg        NLALALNLALNLNLNLNLNLALALNLNLNLALNLNLNLNLALALNL...
g                                                      5238
ab                                                    13654
r                                                      1869
h                                                      3582
X2b                                                     739
X3b                                                      55
hr                                                      437
rbi                                                    1847
sb                                                      138
cs                                                       46
bb                                                     1549
so                                      

Clearly, `sum` is more meaningful for some columns than others. For methods like `mean` for which application to string variables is not just meaningless, but impossible, these columns are automatically exculded:

In [117]:
baseball.mean()

year     2006.92
stint       1.13
g          52.38
ab        136.54
r          18.69
h          35.82
X2b         7.39
X3b         0.55
hr          4.37
rbi        18.47
sb          1.38
cs          0.46
bb         15.49
so         24.08
ibb         1.77
hbp         1.12
sh          1.38
sf          1.20
gidp        3.54
dtype: float64

The important difference between NumPy's functions and Pandas' methods is that the latter have built-in support for handling missing data.

In [118]:
bacteria2

phylum
Firmicutes           NaN
Proteobacteria     632.0
Actinobacteria    1638.0
Bacteroidetes      569.0
dtype: float64

In [119]:
bacteria2.mean()

946.33333333333337

Sometimes we may not want to ignore missing values, and allow the `nan` to propagate.

In [120]:
bacteria2.mean(skipna=False)

nan

Passing `axis=1` will summarize over rows instead of columns, which only makes sense in certain situations.

In [121]:
extra_bases = baseball[['X2b','X3b','hr']].sum(axis=1)
extra_bases.sort_values(ascending=False)

id
88653    69
89439    57
89361    56
89462    55
89396    54
89489    54
89360    54
89371    50
89378    46
89374    46
89389    45
89523    44
89521    42
89463    41
89466    40
88652    39
89438    36
89330    35
89533    33
89481    29
89430    26
89398    26
89347    25
89530    22
89473    20
89495    18
89464    17
89482    16
89499    15
89352    15
         ..
89498     0
89411     0
89525     0
89526     0
88650     0
88649     0
88645     0
88643     0
89341     0
89345     0
89381     0
89493     0
89450     0
89451     0
89372     0
89452     0
89370     0
89460     0
89367     0
89465     0
89384     0
89363     0
89445     0
89388     0
89359     0
89355     0
89354     0
89480     0
89348     0
89420     0
Length: 100, dtype: int64

A useful summarization that gives a quick snapshot of multiple statistics for a `Series` or `DataFrame` is `describe`:

In [122]:
baseball.describe()

year       stint           g          ab          r           h  \
count   100.00000  100.000000  100.000000  100.000000  100.00000  100.000000   
mean   2006.92000    1.130000   52.380000  136.540000   18.69000   35.820000   
std       0.27266    0.337998   48.031299  181.936853   27.77496   50.221807   
min    2006.00000    1.000000    1.000000    0.000000    0.00000    0.000000   
25%    2007.00000    1.000000    9.500000    2.000000    0.00000    0.000000   
50%    2007.00000    1.000000   33.000000   40.500000    2.00000    8.000000   
75%    2007.00000    1.000000   83.250000  243.750000   33.25000   62.750000   
max    2007.00000    2.000000  155.000000  586.000000  107.00000  159.000000   

              X2b         X3b          hr        rbi          sb          cs  \
count  100.000000  100.000000  100.000000  100.00000  100.000000  100.000000   
mean     7.390000    0.550000    4.370000   18.47000    1.380000    0.460000   
std     11.117277    1.445124    7.975537   28.34793    3.694878    1.067613   
min      0.000000    0.000000    0.000000    0.00000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.00000    0.000000    0.000000   
50%      1.000000    0.000000    0.000000    2.00000    0.000000    0.000000   
75%     11.750000    1.000000    6.000000   27.00000    1.000000    0.000000   
max     52.000000   12.000000   35.000000   96.00000   22.000000    6.000000   

               bb          so         ibb        hbp          sh          sf  \
count  100.000000  100.000000  100.000000  100.00000  100.000000  100.000000   
mean    15.490000   24.080000    1.770000    1.12000    1.380000    1.200000   
std     25.812649   32.804496    5.042957    2.23055    2.919042    2.035046   
min      0.000000    0.000000    0.000000    0.00000    0.000000    0.000000   
25%      0.000000    1.000000    0.000000    0.00000    0.000000    0.000000   
50%      1.000000    7.000000    0.000000    0.00000    0.000000    0.000000   
75%     19.250000   37.250000    1.250000    1.00000    1.000000    2.000000   
max    132.000000  134.000000   43.000000   11.00000   14.000000    9.000000   

             gidp  
count  100.000000  
mean     3.540000  
std      5.201826  
min      0.000000  
25%      0.000000  
50%      1.000000  
75%      6.000000  
max     21.000000

`describe` can detect non-numeric data and sometimes yield useful information about it.

In [123]:
baseball.player.describe()

count           100
unique           82
top       coninje01
freq              2
Name: player, dtype: object

We can also calculate summary statistics *across* multiple columns, for example, correlation and covariance.

$$cov(x,y) = \sum_i (x_i - \bar{x})(y_i - \bar{y})$$

In [124]:
baseball.hr.cov(baseball.X2b)

69.076464646464558

$$corr(x,y) = \frac{cov(x,y)}{(n-1)s_x s_y} = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2 \sum_i (y_i - \bar{y})^2}}$$

In [125]:
baseball.hr.corr(baseball.X2b)

0.77906151825397429

In [126]:
baseball.ab.corr(baseball.h)

0.99421740362723732

Try running `corr` on the entire `baseball` DataFrame to see what is returned:

In [127]:
# Write answer here